# 1. Classification: 
Classification is to identify which category a new observation belongs to, on the basis of a training dataset. There are five datasets. For each dataset, we provide the training dataset, training label, and test dataset. Please use the training dataset and training label to build your classifier and predict the test label. A class label is represented by an integer. For example, in the 1st dataset, there are 4 classes where 1 represents the 1st class, 2 represents the 2nd class, etc. Note that, there exist some missing values in some of the dataset (a missing entry is filled by 1.00000000000000e+99), please fill the missing values before perform your classification algorithm.

TrainData 1 contains 3312 features with 150 samples. Testdata1 contains 3312 features with 53 samples. There are 5 classes in this dataset.

TrainData 2 contains 9182 features with 100 samples. Testdata2 contains 9182 features with 74 samples. There are 11 classes in this dataset.

TrainData 3 contains 13  features with 6300 samples. Testdata3 contains 13 features with 2693 samples. There are 9 classes in this dataset.

TrainData 4 contains 112 features with 2547 samples. Testdata4 contains 112 features with 1092 samples. There are 9 classes in this dataset.

TrainData 5 contains 11 features with 1119 samples. Testdata5 contains 11 features with 480 samples. There are 6 classes in this dataset.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/test-data-1/MultLabelTrainData.txt
/kaggle/input/test-data-1/TrainData5.txt
/kaggle/input/test-data-1/TrainData4.txt
/kaggle/input/test-data-1/TrainLabel3.txt
/kaggle/input/test-data-1/TestData4.txt
/kaggle/input/test-data-1/TrainLabel4.txt
/kaggle/input/test-data-1/MissingData2.txt
/kaggle/input/test-data-1/TestData6.txt
/kaggle/input/test-data-1/TrainData3.txt
/kaggle/input/test-data-1/MissingData1.txt
/kaggle/input/test-data-1/TrainData6.txt
/kaggle/input/test-data-1/TrainData1.txt
/kaggle/input/test-data-1/TestData5.txt
/kaggle/input/test-data-1/TestData3.txt
/kaggle/input/test-data-1/TestData1.txt
/kaggle/input/test-data-1/TrainLabel6.txt
/kaggle/input/test-data-1/MissingData3.txt
/kaggle/input/test-data-1/MultLabelTestData.txt
/kaggle/input/test-data-1/TestData2.txt
/kaggle/input/test-data-1/TrainLabel5.txt
/kaggle/input/test-data-1/TrainData2.txt
/kaggle/input/test-data-1/TrainLabel1.txt
/kaggle/input/test-data-1/MultLabelTrainLabel.txt
/kaggle/input/test-data-1/Tr

In [2]:
# Importing Libraries
import numpy as np
import pandas as pd
import random
from random import seed
from random import randrange
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [3]:
# Importing the datasets
traindata1 = pd.read_csv('/kaggle/input/test-data-1/TrainData1.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata2 = pd.read_csv('/kaggle/input/test-data-1/TrainData2.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata3 = pd.read_csv('/kaggle/input/test-data-1/TrainData3.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata4 = pd.read_csv('/kaggle/input/test-data-1/TrainData4.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
traindata5 = pd.read_csv('/kaggle/input/test-data-1/TrainData5.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
train_data = [traindata1, traindata2, traindata3, traindata4, traindata5]

testdata1 = pd.read_csv('/kaggle/input/test-data-1/TestData1.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
testdata2 = pd.read_csv('/kaggle/input/test-data-1/TestData2.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
testdata3 = pd.read_csv('/kaggle/input/test-data-1/TestData3.txt', sep=',', header=None, na_values=1000000000)
testdata4 = pd.read_csv('/kaggle/input/test-data-1/TestData4.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
testdata5 = pd.read_csv('/kaggle/input/test-data-1/TestData5.txt', sep='\s+', header=None, na_values='1.00000000000000e+99')
test_data = [testdata1, testdata2, testdata3, testdata4, testdata5]

trainlabel1 = pd.read_csv('/kaggle/input/test-data-1/TrainLabel1.txt', sep='\t', header=None)
trainlabel2 = pd.read_csv('/kaggle/input/test-data-1/TrainLabel2.txt', sep='\t', header=None)
trainlabel3 = pd.read_csv('/kaggle/input/test-data-1/TrainLabel3.txt', sep='\t', header=None)
trainlabel4 = pd.read_csv('/kaggle/input/test-data-1/TrainLabel4.txt', sep='\t', header=None)
trainlabel5 = pd.read_csv('/kaggle/input/test-data-1/TrainLabel5.txt', sep='\t', header=None)
train_label = [trainlabel1, trainlabel2, trainlabel3, trainlabel4, trainlabel5]

In [4]:
# count the numbers of coloumns where the value = 1.00000000000000e+99
def count_na(data, type):
    print("Missing values in the " + type + " data:")
    for i in range(len(data)):
        missing_val_count = data[i].isnull().sum().sum()
        print( type + " dataset " + str(i+1) + ": " + str(missing_val_count))

count_na(train_data, "train")
count_na(test_data, "test")



Missing values in the train data:
train dataset 1: 9936
train dataset 2: 0
train dataset 3: 1886
train dataset 4: 0
train dataset 5: 0
Missing values in the test data:
test dataset 1: 7021
test dataset 2: 0
test dataset 3: 808
test dataset 4: 0
test dataset 5: 0


In [5]:
# Now we need to replace the missing values in the dataset using the mean of each column
def replace_na(data):
    for i in range(len(data)):
        for column in data[i].columns:   
                data[i][column] = data[i][column].fillna(data[i][column].mean())
         
replace_na(train_data)

# Replacing test dataset 1 missing values
for column in testdata1.columns:
    testdata1[column] = testdata1[column].fillna(testdata1[column].mean())

# Now checking if any remaining missing vals left
count_na(train_data, "train")
count_na(test_data, "test")


Missing values in the train data:
train dataset 1: 0
train dataset 2: 0
train dataset 3: 0
train dataset 4: 0
train dataset 5: 0
Missing values in the test data:
test dataset 1: 0
test dataset 2: 0
test dataset 3: 808
test dataset 4: 0
test dataset 5: 0


In [6]:
# For all the train data, remove the columns with low variance
def remove_low_variance(data):
    for i in range(len(data)):
        selector = VarianceThreshold(threshold=.8 * (1 - .8))
        data[i] = selector.fit_transform(data[i])

remove_low_variance(train_data)


In [7]:
# remove correlated features
def remove_correlation(data, threshold):
    for i in range(len(data)):
        data[i] = pd.DataFrame(data[i])
        col_corr = set()  # Set of all the names of correlated columns
        corr_matrix = data[i].corr()
        for k in range(len(corr_matrix.columns)):
            for j in range(k):
                if abs(corr_matrix.iloc[k, j]) > threshold:
                    colname = corr_matrix.columns[j]
                    col_corr.add(colname)

        data[i] = data[i].drop(data[i].columns[list(col_corr)], axis=1)

remove_correlation(train_data, 0.8)

In [8]:
# get labels for each dataset
def get_unique_values(label):
    unique_values = []
    for i in range(len(label)):
        unique_values.append(pd.unique(label[i][0].values).tolist())
    return unique_values

labels = get_unique_values(train_label)    

print(labels)

[[1, 2, 4, 3, 5], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [9, 1, 8, 6, 2, 4, 7, 5, 3], [1, 2, 3, 4, 5, 6, 7, 8, 9], [5, 6, 7, 4, 8, 3]]


In [9]:
# selected features for each dataset using selectkbest
# la = [1,2,3,4,5,6,7,8,9]
# def select_features(data, labels):
#     threshold = 0.8
#     selected_features = []
#     for label in labels:
#         selector = SelectKBest(chi2, k='all')
#         selector.fit(data, label)
#         selected_features.append(list(selector.scores_))
#     selected_features = np.mean(selected_features, axis=0) > threshold
#     print(selected_features)
#     selected_features = np.max(selected_features, axis=0) > threshold
#     print(selected_features)

# select_features(train_data[2], la)


In [10]:


# Splitting the train data into training and testing
X_train = []
X_test = []
y_train = []
y_test = []

def split_data(data, label):
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state = 21)
    return X_train, X_test, y_train, y_test

for i in range(len(train_data)):
    X_train_i, X_test_i, y_train_i, y_test_i = split_data(train_data[i], train_label[i])
    X_train.append(X_train_i)
    X_test.append(X_test_i)
    y_train.append(y_train_i)
    y_test.append(y_test_i)
    print("Train dataset " + str(i+1) + ": " + str(len(X_train_i)))
    print("Test dataset " + str(i+1) + ": " + str(len(X_test_i)))

Train dataset 1: 105
Test dataset 1: 45
Train dataset 2: 70
Test dataset 2: 30
Train dataset 3: 4410
Test dataset 3: 1890
Train dataset 4: 1782
Test dataset 4: 765
Train dataset 5: 783
Test dataset 5: 336


In [11]:

class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, val=None):
        self.feature_index = feature_index 
        self.threshold = threshold 
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.val = val

In [12]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])

        leaf_val = self.calculate_leaf_value(y)
        return Node(val=leaf_val)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="gini"):
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            # use entropy
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        
        if not tree:
            tree = self.root

        if tree.val is not None:
            print(tree.val)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        
        if tree.val!=None: return tree.val
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


In [13]:

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=5)


classifier.fit(X_train[0], y_train[0])
predictions = classifier.predict(X_test[0].values)
train_predictions_zero = classifier.predict(X_train[0].values)
print("Accuracy for classifier 0:", accuracy_score(y_test[0], predictions))
print("Training Accuracy for classifier 0:", accuracy_score(y_train[0], train_predictions_zero))
classifier.fit(X_train[1], y_train[1])
predictions = classifier.predict(X_test[1].values)
train_predictions_one = classifier.predict(X_train[1].values)
print("Accuracy for classifier 1:", accuracy_score(y_test[1], predictions))
print("Training Accuracy for classifier 1:", accuracy_score(y_train[1], train_predictions_one))
classifier.fit(X_train[2], y_train[2])
predictions = classifier.predict(X_test[2].values)
train_predictions_two = classifier.predict(X_train[2].values)
print("Accuracy for classifier 2:", accuracy_score(y_test[2], predictions))
print("Training Accuracy for classifier 2:", accuracy_score(y_train[2], train_predictions_two))
classifier.fit(X_train[3], y_train[3])
predictions = classifier.predict(X_test[3].values)
train_predictions_three = classifier.predict(X_train[3].values)
print("Accuracy for classifier 3:", accuracy_score(y_test[3], predictions))
print("Training Accuracy for classifier 3:", accuracy_score(y_train[3], train_predictions_three))
classifier.fit(X_train[4], y_train[4])
predictions = classifier.predict(X_test[4].values)
train_predictions_four = classifier.predict(X_train[4].values)
print("Accuracy for classifier 4:", accuracy_score(y_test[4], predictions))
print("Training Accuracy for classifier 4:", accuracy_score(y_train[4], train_predictions_four))

    

Accuracy for classifier 0: 0.8444444444444444
Training Accuracy for classifier 0: 1.0
Accuracy for classifier 1: 0.5
Training Accuracy for classifier 1: 0.7857142857142857
Accuracy for classifier 2: 0.3386243386243386
Training Accuracy for classifier 2: 0.3653061224489796
Accuracy for classifier 3: 0.5751633986928104
Training Accuracy for classifier 3: 0.654320987654321
Accuracy for classifier 4: 0.5416666666666666
Training Accuracy for classifier 4: 0.6832694763729247
